# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm
import scipy.stats as stats

In [2]:
# manually search perceptron hyperparameters for binary classification
import cProfile as profile
import pstats

In [3]:
# --- Response ---
from ev_ssm import ev_ssm
import numpy as np

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.5, tt_var=0.2, tt_lb=0, tt_ub=1,
             ict_off=False, ecc_off=False)
sse.load_A("Aest.csv")

evd0 = sse.ev.copy()

# quiet some EVs by force
# np.random.seed(sse.config["seed"])
# ict_rid = np.random.randint(low=0,
#                             high=sse.ev.shape[0],
#                             size=3000)

# sse.ev.na.iloc[ict_rid] = sse.ev.nam.iloc[ict_rid]
# sse.ev.lc.iloc[ict_rid] = 1
# sse.g_u()

# initial EV data
evd0 = sse.ev.copy()

sse.g_frc()

EVA: ts=18[H], 10000 EVs, Total Q=87.7 MWh
Online 2925, Q=73.03 MWh, SoC=0.7717
Power(MW): Pt=-7.9463, Pc=-7.9463, Pd=-0.0
Ctrl: 0=1584; 1=1341; 
EVA: Load A from Aest.csv.


[23.844112764286482, 1.82771328502618]

In [4]:
sse.ev['lc']

0       0
1       0
2       0
3       0
4       0
       ..
3505    0
3506    0
3507    0
3508    0
3509    0
Name: lc, Length: 3510, dtype: int64

In [5]:
self = sse
self.ev[self.ev["na"].values >= self.ev["nam"].values].index


Int64Index([  35,   37,   48,   59,   76,  119,  149,  223,  305,  334,  384,
             441,  454,  468,  542,  558,  627,  674,  677,  757,  782,  789,
             840,  853,  919,  969, 1018, 1095, 1196, 1236, 1302, 1307, 1373,
            1374, 1398, 1436, 1442, 1560, 1621, 1624, 1731, 1749, 1769, 1804,
            1810, 1890, 1897, 1980, 2002, 2003, 2024, 2033, 2076, 2125, 2192,
            2206, 2217, 2324, 2420, 2421, 2480, 2487, 2528, 2542, 2563, 2643,
            2646, 2721, 2739, 2756, 2781, 2799, 2816, 2831, 2874, 2937, 2962,
            2982, 3006, 3062, 3100, 3142, 3144, 3164, 3206, 3221, 3230, 3255,
            3272, 3298, 3380, 3398],
           dtype='int64')

In [7]:
self.ev.loc[[1,2,3], ['na', 'nam']]

,na,nam
1,2134,3483
2,2183,2873
3,1927,2932


In [6]:
self.ev["na"]

0       1996
1       2134
2       2183
3       1927
4       1982
        ... 
3505    1776
3506    1602
3507    1923
3508    1756
3509    1646
Name: na, Length: 3510, dtype: int64

In [ ]:
prof = profile.Profile()
prof.enable()
for i in range(20):
        sse.run(tf=18 + i/3600, Pi=0.5*i,
                is_updateA=False, is_rstate=True,
                is_test=False, disable=True,
                )
for i in range(20):
        sse.run(tf=18 + (20 + i)/3600, Pi=10 - 0.5*i,
                is_updateA=False, is_rstate=True,
                is_test=False, disable=True)
prof.disable()

In [ ]:
sse.plot_agc()

In [ ]:
sse.ev.iloc[0]

In [ ]:
stats = pstats.Stats(prof).strip_dirs().sort_stats("tottime")
stats.print_stats(20)

In [ ]:
sse.run(tf=18 + 10/3600, Pi=0,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

# sse.run(tf=18 + 100/3600, Pi=0,
#         is_updateA=False, is_rstate=True,
#         is_test=False, disable=False)

In [ ]:
sse.plot_agc()

In [ ]:
sse.plot()